# Case Study 1 : Collecting Data from Twitter

# Problem 1: Sampling Twitter Data with Streaming API about a Certain Topic

* Select and include a case topic that you are interested in, for example, "WPI" or "Lady Gaga" or COVID-19.
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. It would be recommended that the number of tweets should be larger than 1000, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [ ]:
import twitter

def oauth_login():
    # XXX: Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'OEBQxusAmLaJIP2o3WQTCNxBA'
    CONSUMER_SECRET ='rA29WDveS9w50jF9Ke0edA8xuS2oTcehi2Bq6J11cHPwR4RpQE'
    OAUTH_TOKEN = '1492269919924195329-KdGOMwMHOJZFKTRvm3ha0zysLcg9ev'
    OAUTH_TOKEN_SECRET = 'LQJTZ8HNc5qWleTxhUZYfAy37VJmKH75L9h9OFI2cCRZ0'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

# Sample usage
twitter_api = oauth_login()    

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)
import flask
print(flask.__version__)

1.1.2


In [ ]:
import json
import time

def twitter_search(twitter_api, q, max_results=1000, **kw):

    # See https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    # and https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators
    # for details on advanced search criteria that may be useful for 
    # keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100,lang='en', **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://developer.twitter.com/en/docs/basics/rate-limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

# Sample usage

twitter_api = oauth_login()

q = 'Beijing2022'
results = twitter_search(twitter_api, q, max_results=1000)
        
# Show one sample search result by slicing the list...
data = json.dumps(results, indent=1)

file = open('extracted_tweets_beijing_2022_final.txt', 'w')
file.write(data)
file.close()

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [ ]:
from collections import Counter
from prettytable import PrettyTable
import nltk

tweet_texts = [ tweet['text'] 
                 for tweet in results ]
words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split()
                 if word not in ['RT', '&amp;', '|', '-'] and '#' not in word and '@' not in word# filter out RT and ampersand
        ]

# Use the natural language toolkit to eliminate stop words

# nltk.download('stopwords') # download stop words if you do not have it
stop_words = nltk.corpus.stopwords.words('english')
stop_words_french = nltk.corpus.stopwords.words('french')
stop_words_spanish = nltk.corpus.stopwords.words('spanish')
non_stop_words = [w for w in words if w.lower() not in stop_words + stop_words_french + stop_words_spanish]

# frequency of words
count = Counter(non_stop_words).most_common()

# table of the top 30 words with their counts
pretty_table = PrettyTable(field_names=['Word', 'Count']) 
[ pretty_table.add_row(w) for w in count[:30] ]
pretty_table.align['Word'] = 'l'
pretty_table.align['Count'] = 'r'
print("Number of Tweets: " + str(len(tweet_texts)))
print(pretty_table)

Number of Tweets: 1100
+------------+-------+
| Word       | Count |
+------------+-------+
| Olympic    |   166 |
| Winter     |   135 |
| figure     |   132 |
| skating    |    80 |
| medals     |    74 |
| Yuzuru     |    71 |
| gold       |    70 |
| Olympics   |    66 |
| first      |    61 |
| one        |    57 |
| together   |    56 |
| Gala       |    56 |
| home       |    55 |
| medal      |    55 |
| Skating    |    55 |
| Figure     |    54 |
| Wearing    |    53 |
| hats       |    53 |
| performing |    53 |
| Hany…      |    53 |
| team       |    52 |
| became     |    47 |
| flight     |    46 |
| Games      |    46 |
| athletes   |    46 |
| Extra      |    45 |
| baggage    |    45 |
| needed     |    45 |
| today…     |    45 |
| 🥇🥈       |    45 |
+------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [ ]:
# Create a list of all tweets with a retweeted_status key, and index the originator of that tweet and the text.
retweets = [
            (tweet['retweet_count'],
             tweet['text'])
            for tweet in results                      
            ]

#del list
rtdict = dict(list(zip(tweet_texts, retweets)))
def sortRTdict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux
sortedrtdict = sortRTdict(rtdict)
rts = []
tweets = []

for duo in sortedrtdict[:10]:
    tweets.append(duo[1])
    rts.append(duo[0])

pretty_table = PrettyTable(field_names = ['Retweets','Text'])

# Sort tweets by descending number of retweets and display the top 10 results in a table.
[pretty_table.add_row(row) for row in rts]
pretty_table.max_width['Text'] = 200
pretty_table.align = 'l'
print(pretty_table)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------+
| Retweets | Text                                                                                                                                         |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------+
| 14597    | RT @Olympics: When @weareoneEXO rocked the PyeongChang2018 #ClosingCeremony 😍 🎶🎤                                                          |
|          |                                                                                                                                              |
|          | Can't wait for tonight's ceremony! 🤩                                                                                                        |
|          |                                                        

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags and top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:
# Extract the screen names which appear among the collection of tweets
screen_names = [user_mention['screen_name']
               for tweet in results
                   for user_mention in tweet['entities']['user_mentions']]

# Extract the hashtags which appear among the collection of tweets
hashtags = [ hashtag['text']
           for tweet in results
               for hashtag in tweet['entities']['hashtags']]

# Simultaneously determine the frequency of screen names/hashtags, and display the top 10 most common in a table.
for label, data in (('Screen Name',screen_names),
                   ('Hashtag',hashtags)):
    pretty_table = PrettyTable(field_names =[label,'Count'])
    counter = Counter(data)
    [ pretty_table.add_row(entity) for entity in counter.most_common()[:10]]
    pretty_table.align[label] ='l'
    pretty_table.align['Count'] = 'r'
    print(pretty_table)

+-----------------+-------+
| Screen Name     | Count |
+-----------------+-------+
| Beijing2022     |   131 |
| Olympics        |   101 |
| TeamGB          |    73 |
| USFigureSkating |    57 |
| SpokespersonCHN |    55 |
| nathanwchen     |    37 |
| billmaher       |    34 |
| nytimes         |    33 |
| CarlZha         |    32 |
| weareoneEXO     |    32 |
+-----------------+-------+
+--------------------------+-------+
| Hashtag                  | Count |
+--------------------------+-------+
| Beijing2022              |   577 |
| BingDwenDwen             |   126 |
| Olympics                 |    85 |
| ClosingCeremony          |    77 |
| WinterOlympics           |    75 |
| TeamGB                   |    54 |
| MedalistMonday           |    37 |
| TogetherForASharedFuture |    34 |
| EXO                      |    32 |
| 엑소                     |    32 |
+--------------------------+-------+


# Problem 3: Getting "All" Friends and "All" Followers of a Popular User in Twitter

* Choose a popular twitter user who has many followers in the collected tweets of your case topic.
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers, and their screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers, and their screen names in a table.

In [ ]:
import tweepy
  
# assign the values accordingly
consumer_key = 'OEBQxusAmLaJIP2o3WQTCNxBA'
consumer_secret = 'rA29WDveS9w50jF9Ke0edA8xuS2oTcehi2Bq6J11cHPwR4RpQE'
access_token = '1492269919924195329-KdGOMwMHOJZFKTRvm3ha0zysLcg9ev'
access_token_secret = 'LQJTZ8HNc5qWleTxhUZYfAy37VJmKH75L9h9OFI2cCRZ0'
  
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)
  
# calling the api 
api = tweepy.API(auth)

name = [ tweet['user']['screen_name'] 
                for tweet in results ]

count = []
for i in range(100):
    user = api.get_user(screen_name= name[i])
    followers_count = user.followers_count
    friends_count = user.friends_count
    count.append(followers_count+friends_count)
top3 = [count.index(x) for x in sorted(count, reverse=True)[:4]]
i = top3[3]
user = api.get_user(screen_name= name[i])

# screen name of the user
print(name[i])
name = name[i]

rodmickleburgh


In [ ]:
# getting the friends list
friends = api.get_friend_ids(screen_name = name, count = 300)

friend_list = []    
for friend in friends:
    friend_list.append(api.get_user(user_id = friend).screen_name)

In [ ]:
print(len(friend_list))

300


In [ ]:
# Plot 20 out of the friend, plot their ID numbers, and their screen names in a table.
dfriend = []
for i in range(20):
    dfriend.append([friends[i],friend_list[i]])
dfri = pd.DataFrame(dfriend,columns=['ID', 'Screen name'])
print(dfri)

                     ID      Screen name
0             593175545        missprill
1             440389866      DavidSpaner
2             130120037      PippaCrerar
3              56556395         ann_rees
4   1015339707343835136    Trish09071900
5             299820130   TriciaLockwood
6             385229000       MJSchulman
7             159128818   lynnmilesmusic
8             170336862      rosecousins
9              19657809       maxfawcett
10  1296239665377206273     KathyCalder6
11             18980055        cbcradioq
12           2975475327     dwebsterhist
13            612114921      DanielKalla
14  1194627927272542209  leylahfernandez
15           2252615726        ca_sayers
16             18440301      PennyDaflos
17            380900984       olddanbook
18             18788452            lslot
19            340466229     IAmAmnaNawaz


In [ ]:
# getting the followers list
followers = api.get_follower_ids(screen_name = name, count = 300)

follower_list = []    
for follower in followers:
    follower_list.append(api.get_user(user_id = follower).screen_name)

In [ ]:
import pandas as pd

# Plot 20 out of the followers, plot their ID numbers, and their screen names in a table.
dfollower = []
for i in range(20):
    dfollower.append([followers[i],follower_list[i]])
dfoll = pd.DataFrame(dfollower,columns=['ID', 'Screen name'])
print(dfoll)

                     ID      Screen name
0   1494718822087024645         CBernent
1   1495809219202732038  plumber_another
2            3108888065     hnstCanadian
3             261920529     roguechimp99
4   1494424236756291585  paulmil91849591
5   1486528713718128648  WealthExecutive
6            1279036994    AmandaMechele
7   1485409328026640384          RoryLew
8   1495229426669342724    Baboucarr1111
9            2286487057        AJandTara
10             43053298    Miss_T_tweets
11  1488273192682590210  Dontsweatthesm5
12  1407158788910682114  MarkZuckerpunch
13  1439957711173660672   PapyrusBrigade
14  1489964710111981576  PennyHa98551852
15  1455963692672053250  CityHallWchWchr
16            272969211      NormanRoots
17            792615860    drivercom99in
18  1258762616790331394    10minreviewer
19            375258014  DavidChenTweets


* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [ ]:
df1 = pd.DataFrame(followers,columns=["ID"])
df2 = pd.DataFrame(friends,columns=["ID"])
df3 = df1.merge(df2, how = 'inner' ,indicator=False)

mutual_names = []
for f in df3["ID"]:
    mutual_names.append( api.get_user(user_id = f).screen_name)

mutuals = []
for i in range(len(df3["ID"])):
    mutuals.append([df3["ID"][i], mutual_names[i]])
mutualsPD = pd.DataFrame(mutuals,columns=['ID', 'Screen name'])
print(mutualsPD)

          ID  Screen name
0  440389866  DavidSpaner


# Problem 4: Domain Question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a question, which Twitter data could help answer from your collected tweets, based upon your case topic of your chosen public organization, private company, social community, etc., in a domain.
* How could Twitter data help that company/organization/community spend its resources to answer the above question?

In [ ]:
countries = [ tweet['user']['location'] 
                for tweet in results ]

# frequency of countries
count = Counter(countries).most_common()

# table of the top 30 words with their counts
pretty_table = PrettyTable(field_names=['Location', 'Count']) 
[ pretty_table.add_row(w) for w in count[1:4] ]
pretty_table.align['Word'] = 'l'
pretty_table.align['Count'] = 'r'
print(pretty_table)

+------------------+-------+
|     Location     | Count |
+------------------+-------+
| West Newbury, MA |    26 |
|    Australia     |    22 |
|  United Kingdom  |    11 |
+------------------+-------+
